<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 603.8 MB 6.8 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [2]:
!git clone https://github.com/raymondwcs/learning_spacy/

fatal: destination path 'learning_spacy' already exists and is not an empty directory.


In [3]:
import spacy

sentences = [
  '大巴人以為跌一日博明天升',  #pos
  '入貨喇喂',  #pos
  '盡快掃貨',  #pos
  '樓主有料……聽日港股會升',  #pos
  '預咗侵侵佢輸硬,拜登上場,中美貿易戰結束,美跌港升',   #pos
  '呢幾日狂升',  #pos
  '大調整，股災危險', #neg
  '開始睇淡走貨啦', #neg
  '因為要殺熊', #neg
  '無人會聽你講 直至欲哭無淚為止', #neg
  '可以死守嗎', #neg
  '唔投降就等仆街了', #neg
]

In [4]:
nlp = spacy.load('zh_core_web_lg')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")
nlp.tokenizer.pkuseg_update_user_dict(["大巴人", "博", "投降"])

all_list = []
for sentence in sentences:
  doc = nlp(sentence)
  list = ''
  for token in doc:
    list += token.text + ' '
  all_list.append(list)

print(all_list)

['大巴人 以為 跌 一日 博 明天 升 ', '入貨 喇喂 ', '盡快 掃貨 ', '樓主 有 料 …… 聽日 港股 會 升 ', '預咗 侵侵 佢 輸硬 , 拜登 上場 , 中美 貿易戰 結束 , 美跌 港升 ', '呢 幾 日 狂升 ', '大 調整 ， 股災 危險 ', '開始 睇淡 走貨 啦 ', '因為 要 殺熊 ', '無 人 會 聽 你 講 直至 欲 哭 無淚為止 ', '可以 死守 嗎 ', '唔 投降 就 等 仆街 了 ']


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF
tfidf = TfidfVectorizer()
weight = tfidf.fit_transform(all_list).toarray()
word = tfidf.get_feature_names()

# for i in range(len(weight)):
#   print('\nsentence {}'.format(i))
#   for j in range(len(word)):
#     print(word[j], weight[i][j])

df = pd.DataFrame(weight,columns=word)
print('Features: {}'.format(df.columns))
print('Number of features: {}'.format(len(df.columns)))
df

Features: Index(['一日', '上場', '中美', '仆街', '以為', '侵侵', '入貨', '危險', '可以', '喇喂', '因為', '大巴人',
       '投降', '拜登', '掃貨', '明天', '樓主', '死守', '殺熊', '港升', '港股', '無淚為止', '狂升',
       '盡快', '直至', '睇淡', '結束', '美跌', '聽日', '股災', '調整', '貿易戰', '走貨', '輸硬', '開始',
       '預咗'],
      dtype='object')
Number of features: 36


,一日,上場,中美,仆街,以為,侵侵,入貨,危險,可以,喇喂,因為,大巴人,投降,拜登,掃貨,明天,樓主,死守,殺熊,港升,港股,無淚為止,狂升,盡快,直至,睇淡,結束,美跌,聽日,股災,調整,貿易戰,走貨,輸硬,開始,預咗
0,0.5,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.5,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.707107,0.00000,0.000000,0.707107,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
2,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.707107,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.707107,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
3,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.57735,0.000000,0.000000,0.000000,0.57735,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
4,0.0,0.316228,0.316228,0.000000,0.0,0.316228,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.316228,0.000000,0.0,0.00000,0.000000,0.000000,0.316228,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.316228,0.316228,0.00000,0.00000,0.00000,0.316228,0.00000,0.316228,0.00000,0.316228
5,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,1.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
6,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.57735,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.57735,0.57735,0.000000,0.00000,0.000000,0.00000,0.000000
7,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.57735,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.57735,0.000000,0.57735,0.000000
8,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.707107,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.707107,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
9,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.707107,0.0,0.000000,0.707107,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000


In [38]:
top_N = 3
for i in range(len(weight)):  # weights of each sentence
  sorted = np.argsort(weight[i])[::-1]  # reverse
  sorted = sorted[0:top_N]
  top_features = [word[i] for i in sorted]
  print('Sentence {}: {}'.format(i,top_features))

Sentence 0: ['一日', '明天', '以為']
Sentence 1: ['喇喂', '入貨', '預咗']
Sentence 2: ['掃貨', '盡快', '預咗']
Sentence 3: ['港股', '聽日', '樓主']
Sentence 4: ['預咗', '中美', '港升']
Sentence 5: ['狂升', '預咗', '樓主']
Sentence 6: ['調整', '股災', '危險']
Sentence 7: ['開始', '睇淡', '走貨']
Sentence 8: ['殺熊', '因為', '樓主']
Sentence 9: ['無淚為止', '直至', '預咗']
Sentence 10: ['死守', '可以', '上場']
Sentence 11: ['仆街', '投降', '預咗']
